In [1]:
import json
import numpy as np
import pandas as pd
import re
import glob
from dotenv import load_dotenv
import boto3

from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

import os

load_dotenv() # Load the environment variables

API_KEY = os.getenv("DG_API_KEY")


cwd = os.getcwd() # Get the current working directory (cwd)
s3 = boto3.client('s3') # Create an S3 client
deepgram = DeepgramClient(API_KEY)

In [2]:
filenames = glob.glob('../pilot/*.scienceData.jsonl')
data = []
for filename in filenames:
    with open(filename) as f:
        for line in f:
            parsed = json.loads(line)
            if parsed['sampleId'] != 'missing':
                data.append(parsed)

sorted_data = sorted(data, key=lambda x: x['treatment']['name']+x['gameId'])

new_data = []
game_lookups = {}
game_index = 0
for i, row in enumerate(sorted_data):
    row["player"] = i
    if row["gameId"] not in game_lookups:
        game_lookups[row["gameId"]] = game_index
        row["game"] = game_index
        game_index += 1
    else:
        row["game"] = game_lookups[row["gameId"]]

    if len(row["speakerEvents"]) > 0: # if they joined the discussion only
        new_data.append(row)

data = new_data
index = pd.MultiIndex.from_tuples([(row['treatment']["name"], row['game'], int(row['position'])) for row in data], names=['treatment', 'game', 'position'])
len(data)

21

In [3]:
def getFileNames(participant, s3, s3_bucket, s3_region):
    # get S3 filenames for all videos recorded for this participant's group
    folder = f"deliberation/{participant['recordingsFolder']}"
    remote_paths = s3.list_objects_v2(Bucket=s3_bucket, Prefix=folder)['Contents']
    return remote_paths

def getMetadata(local_files):
    # get metadata for all files downloaded for this participant's group
    collector = []
    for filepath in local_files:
        
        basename = os.path.basename(filepath)

        # get start time from metadata
        output = ! ffprobe $filepath
        start_time = re.search(' start: (\d+\.\d+)', '\n'.join(output))
        start = start_time.group(1) if start_time else None

        if ("video") in basename:
            # get resolution for video
            resolution_cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of csv=s=x:p=0 {filepath}'
            resolution_output = ! $resolution_cmd
            resolution = resolution_output[0].strip()
            resolution_x = int(resolution.split("x")[0])
            track_type = "video"
            
            # get start from first frame
            cmd = f'ffprobe -v 0 -show_entries packet=pts -of compact=p=0:nk=1 -read_intervals 0 -select_streams v {filepath} | head -1'
            output = ! $cmd
            first_frame_time = float(output[0])/1000

            # get time of last frame
            cmd = f'ffprobe -v 0 -show_entries packet=pts -of compact=p=0:nk=1 -read_intervals 999999 -select_streams v {filepath}  | tail -1 '
            output = ! $cmd
            last_frame_time = float(output[0])/1000

        else:
            resolution = np.nan
            resolution_x = np.nan
            track_type = "audio"
            first_frame_time = np.nan
            last_frame_time = np.nan

        collector.append({
            "file": filepath,
            "basename": basename,
            "track_type": track_type,
            "internal_start": float(start),
            "resolution": resolution,
            "resolution_x": resolution_x,
        })

    metadata = pd.DataFrame(collector)
    metadata = metadata.sort_values("internal_start").reset_index(drop=True)

    # which tracks go together?
    video_tracks = metadata[metadata['track_type']=="video"]
    audio_tracks = metadata[metadata['track_type']=="audio"]
    for group, (video_track_index, video_track) in enumerate(video_tracks.iterrows()):
        audio_track_index = (audio_tracks['internal_start'] - video_track['internal_start']).abs().idxmin()
        metadata.loc[video_track_index, 'track_group'] = str(int(group))
        metadata.loc[audio_track_index, 'track_group'] = str(int(group))

    internal_group_starts = metadata.groupby("track_group")['internal_start'].min() # internal time that the first track in the group starts
    metadata['group_internal_start'] = metadata['track_group'].map(internal_group_starts)
    metadata['group_internal_offset'] = metadata['internal_start'] - metadata['group_internal_start']

    return metadata

<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/1703393873.py:16: SyntaxWarning: invalid escape sequence '\d'
  start_time = re.search(' start: (\d+\.\d+)', '\n'.join(output))


In [4]:
def get_participant_recordings_data(participant):
    if participant['recordingIds'] == "missing":
        return None
    
    folder = participant['recordingsFolder']
    s3_bucket = participant['config']['videoStorage']['bucket']
    s3_region = participant['config']['videoStorage']['region']
    
    remote_paths = getFileNames(participant, s3, s3_bucket, s3_region)
    print("remote_paths", remote_paths)

    files = []
    for recordingId in participant['recordingIds']:
        files += [remote_path['Key'] for remote_path in remote_paths if recordingId in remote_path['Key']]

    print("files", files)

    os.makedirs(cwd + "/recordings/" + folder, exist_ok=True)
    local_files = []
    for file in files:
        path = "recordings/" + "/".join(file.split("/")[1:])
        if not path.endswith(".webm"): 
            path += ".webm"
        local_files.append(path)
        if not os.path.exists(path):
            print("downloading to path", path)
            s3.download_file(s3_bucket, file, path)

    metadata = getMetadata(local_files)
    return metadata

In [5]:

def transcribe(audio_file):
    print("transcribing", audio_file)
    
    with open(audio_file, "rb") as file:
        buffer_data = file.read()

    payload: FileSource = {
        "buffer": buffer_data,
    }

    #STEP 2: Configure Deepgram options for audio analysis
    options = PrerecordedOptions(
        model="nova-2",
        utterances=True,
    )

    # STEP 3: Call the transcribe_file method with the text payload and options
    response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

    return response

    

In [10]:
for i, row in get_participant_recordings_data(data[8]).iterrows():
    
    print(row)
    

remote_paths [{'Key': 'deliberation/20240829_1458_Pilot_KZMBPG/1724947347486-0d3257a4-4d34-4149-9a6b-5ed74d80f54c-cam-audio-1724947348970', 'LastModified': datetime.datetime(2024, 8, 29, 16, 2, 30, tzinfo=tzutc()), 'ETag': '"abe70f6a60c2a471e96b112a5e9a70ba-1"', 'Size': 2411356, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240829_1458_Pilot_KZMBPG/1724947347486-0d3257a4-4d34-4149-9a6b-5ed74d80f54c-cam-video-1724947348971', 'LastModified': datetime.datetime(2024, 8, 29, 16, 2, 30, tzinfo=tzutc()), 'ETag': '"0cfc8fec3e8c4b7ba1cf21ed60906960-24"', 'Size': 121098544, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240829_1458_Pilot_KZMBPG/1724947347486-99db951a-efbe-4935-abe1-ec4aa6cf3937-cam-audio-1724947348459', 'LastModified': datetime.datetime(2024, 8, 29, 16, 2, 29, tzinfo=tzutc()), 'ETag': '"456f4e0ffefe84b651bdcb59e2055837-1"', 'Size': 2525125, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240829_1458_Pilot_KZMBPG/1724947347486-99db951a-efbe-4935-abe1-ec4aa6cf393

In [13]:
def make_transcription_files(participant):
    recordings = get_participant_recordings_data(participant)
    position = participant['position']
    if recordings is None:
        return
    # print(recordings[recordings['track_type']=="audio"])

    if(len(recordings[recordings['track_type']=="audio"]['file']) > 0):
        for i, row in recordings[recordings['track_type']=="audio"].iterrows():
            print(row)
            audio_file = row['file']
            internal_start = row['internal_start']
            
            utterances_filepath = audio_file.replace(".webm", "-utterances.csv")
            if os.path.exists(utterances_filepath):
                print("skipping transcription of", audio_file)
                continue

            response = transcribe(audio_file)
            transcription_df = pd.DataFrame(response.to_dict()['results']['utterances'])

            # export utterances csv
            utterances = transcription_df[["start", "end", "confidence", "transcript"]]
            utterances['position'] = position
            utterances['start'] += internal_start # add the start time of the recording
            utterances['end'] += internal_start
            utterances.to_csv(utterances_filepath, index=False)

            # export words csv
            words_list = []
            for j, word_row in transcription_df.iterrows():
                words_list += word_row['words']

            words = pd.DataFrame(words_list)
            words['position'] = position
            words['start'] += internal_start
            words['end'] += internal_start
            words_filepath = audio_file.replace(".webm", "-words.csv")
            words.to_csv(words_filepath, index=False)


In [14]:
completed = []
for i, participant in enumerate(data):
    if participant['player'] in completed:
        continue
    print(i, participant['player'])

    make_transcription_files(participant)
    completed.append(participant['player'])


0 0
remote_paths [{'Key': 'deliberation/20240829_1458_Pilot_BGAB07/1724947313451-210e4d0e-38e7-4348-9f70-68d8110a122c-cam-audio-1724947314491', 'LastModified': datetime.datetime(2024, 8, 29, 16, 1, 56, tzinfo=tzutc()), 'ETag': '"89a063c93b25a8dd4e43c4b06d079eb9-1"', 'Size': 1014316, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240829_1458_Pilot_BGAB07/1724947313451-210e4d0e-38e7-4348-9f70-68d8110a122c-cam-video-1724947314494', 'LastModified': datetime.datetime(2024, 8, 29, 16, 1, 56, tzinfo=tzutc()), 'ETag': '"7e4803e7fe088380c8868c089bdcb938-12"', 'Size': 61066427, 'StorageClass': 'STANDARD'}]
files ['deliberation/20240829_1458_Pilot_BGAB07/1724947313451-210e4d0e-38e7-4348-9f70-68d8110a122c-cam-audio-1724947314491', 'deliberation/20240829_1458_Pilot_BGAB07/1724947313451-210e4d0e-38e7-4348-9f70-68d8110a122c-cam-video-1724947314494']
file                     recordings/20240829_1458_Pilot_BGAB07/17249473...
basename                 1724947313451-210e4d0e-38e7-4348-9f70-68d8110a.

/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/359421643.py:18: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


4 4
remote_paths [{'Key': 'deliberation/20240815_2018_miniPiAKNVYX/1723755710029-0591956c-a9d3-4480-82c4-ed298a17dec2-cam-audio-1723755710431', 'LastModified': datetime.datetime(2024, 8, 15, 21, 1, 51, tzinfo=tzutc()), 'ETag': '"4982f1c28c7240c9e40482aa395c78e5-1"', 'Size': 2189670, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240815_2018_miniPiAKNVYX/1723755710029-0591956c-a9d3-4480-82c4-ed298a17dec2-cam-video-1723755710433', 'LastModified': datetime.datetime(2024, 8, 15, 21, 1, 51, tzinfo=tzutc()), 'ETag': '"0b28647d91f42c5bc46c0b6aa5cf853c-6"', 'Size': 29061389, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240815_2018_miniPiAKNVYX/1723755710029-10933107-59d8-46eb-b78c-08f8f41bca7a-cam-audio-1723755716888', 'LastModified': datetime.datetime(2024, 8, 15, 21, 1, 58, tzinfo=tzutc()), 'ETag': '"88b4499927024bdfeeacb7f3f5fcfe14-1"', 'Size': 2484298, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240815_2018_miniPiAKNVYX/1723755710029-10933107-59d8-46eb-b78c-08f8f41bc

/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterances['position'] = position
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterances['start'] += internal_start # add the start time of the recording
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:26: SettingWithCopyWarning: 
A value is trying to be set on a cop

file                     recordings/20240815_2018_miniPiAKNVYX/17237557...
basename                 1723755710029-0591956c-a9d3-4480-82c4-ed298a17...
track_type                                                           audio
internal_start                                                       0.385
resolution                                                             NaN
resolution_x                                                           NaN
track_group                                                              0
group_internal_start                                                 0.385
group_internal_offset                                                  0.0
Name: 0, dtype: object
transcribing recordings/20240815_2018_miniPiAKNVYX/1723755710029-0591956c-a9d3-4480-82c4-ed298a17dec2-cam-audio-1723755710431.webm


...

/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/359421643.py:18: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


20 20
remote_paths [{'Key': 'deliberation/20240904_1519_Pilot_8YRK6A/1725467480299-16c2892f-2a60-4cc7-8eb5-ac14f83a333e-cam-audio-1725467481270', 'LastModified': datetime.datetime(2024, 9, 4, 16, 31, 22, tzinfo=tzutc()), 'ETag': '"8e7869608ec7a9d75d82b19b9463d550-1"', 'Size': 2227249, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240904_1519_Pilot_8YRK6A/1725467480299-16c2892f-2a60-4cc7-8eb5-ac14f83a333e-cam-video-1725467481271', 'LastModified': datetime.datetime(2024, 9, 4, 16, 31, 22, tzinfo=tzutc()), 'ETag': '"e139e8fd7c7c231dcca83e5d76039f69-29"', 'Size': 150607204, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240904_1519_Pilot_8YRK6A/1725467480299-2c8bfb44-f14a-4a27-8573-895fa346dcae-cam-audio-1725467481590', 'LastModified': datetime.datetime(2024, 9, 4, 16, 31, 22, tzinfo=tzutc()), 'ETag': '"a194edc123e666be6d90e9c7e74a4099-1"', 'Size': 2287007, 'StorageClass': 'STANDARD'}, {'Key': 'deliberation/20240904_1519_Pilot_8YRK6A/1725467480299-2c8bfb44-f14a-4a27-8573-895fa

/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterances['position'] = position
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterances['start'] += internal_start # add the start time of the recording
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:26: SettingWithCopyWarning: 
A value is trying to be set on a cop

file                     recordings/20240904_1519_Pilot_8YRK6A/17254674...
basename                 1725467480299-16c2892f-2a60-4cc7-8eb5-ac14f83a...
track_type                                                           audio
internal_start                                                       0.942
resolution                                                             NaN
resolution_x                                                           NaN
track_group                                                              0
group_internal_start                                                 0.942
group_internal_offset                                                  0.0
Name: 0, dtype: object
transcribing recordings/20240904_1519_Pilot_8YRK6A/1725467480299-16c2892f-2a60-4cc7-8eb5-ac14f83a333e-cam-audio-1725467481270.webm


/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/359421643.py:18: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterances['position'] = position
/var/folders/2l/3kmvzs0s1sggnw_9sl0ls8080000gn/T/ipykernel_18431/2514948886.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [15]:
%debug

> /Users/jamesphoughton/github/contact-theory-topic/env/lib/python3.12/site-packages/pandas/core/indexes/base.py(3812)get_loc()
   3810             ):
   3811                 raise InvalidIndexError(key)
-> 3812             raise KeyError(key) from err
   3813         except TypeError:
   3814             # If we have a listlike key, _check_indexing_error will raise



In [26]:
# Stitch together the individual transcription files

# for each of the folders in the recordings directory
# get all the files in the folder that end in -utterances.csv
for folder in os.listdir(cwd + "/recordings"):
    files = glob.glob(cwd + f"/recordings/{folder}/*-audio-*-utterances.csv")
    
    # load all the files into a dataframe
    if len(files) == 0:
        continue
    utterances = pd.concat([pd.read_csv(file) for file in files])

    # sort by start time
    utterances = utterances.sort_values("start")

    # save the dataframe to a csv
    utterances.to_csv(f"{cwd}/recordings/{folder}/group_transcript.csv", index=False)

    # merge consecutive utterances by the same speaker (indicated by "position" column)
    # reset the start and end times to the beginning and end of the merged utterance
    # combine the transcripts in order
    utterances['position'] = utterances['position'].astype(int)
    utterances['position_shifted'] = utterances['position'].shift(1)
    utterances['position_change'] = utterances['position'] != utterances['position_shifted']
    utterances['group'] = utterances['position_change'].cumsum()
    utterances['group_start'] = utterances.groupby('group')['start'].transform('first')
    utterances['group_end'] = utterances.groupby('group')['end'].transform('last')
    utterances['merged_transcript'] = utterances.groupby('group')['transcript'].transform(' '.join)
    utterances = utterances.drop_duplicates('group')
    utterances = utterances.drop(columns=['confidence', 'position_shifted', 'position_change', 'group', 'transcript', 'start', 'end'])
    utterances = utterances.rename(columns={'group_start': 'start', 'group_end': 'end', 'merged_transcript': 'transcript'})
    utterances["conversation_id"] = folder

    




    # save the dataframe to a csv
    utterances.to_csv(f"{cwd}/recordings/{folder}/group_transcript_merged_consecutives.csv", index=False)
    